In [1]:
# %%
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
# exon-specific Δlogit
from helper import (
    load_and_align_for_delta_logit, pull_vectors_from_row,
    delta_logit_scores, curve_score_from_dlogit, hard_preds_from_dlogit,
    binary_metrics, find_contrastive_root, get_prediction_file,pull_pm1_vectors_from_row,
    pull_pm1_vectors_from_row_predfilter,
)
import time

timestamp = time.strftime("%Y%m%d-%H%M%S")

# --- Detect and set CONTRASTIVE_ROOT environment variable ---
root_path = str(find_contrastive_root())
os.environ["CONTRASTIVE_ROOT"] = root_path
print(f"✅ CONTRASTIVE_ROOT set to: {root_path}")

# --- Root Paths ---
ROOT_RESULTS = f"{root_path}/files/results"
DATA_BASE = f"{root_path}/data/TS_data/tabula_sapiens/final_data"

# --- Ground Truth Binary Files ---
# GT_HIGH = f"{DATA_BASE}/variable_cassette_exons_with_binary_labels_HIGH_TissueBinPsi.csv"
# GT_LOW  = f"{DATA_BASE}/variable_cassette_exons_with_binary_labels_LOW_TissueBinPsi.csv"

# --- Output directory ---
OUT_DIR = f"{ROOT_RESULTS}/../classification_eval"
os.makedirs(OUT_DIR, exist_ok=True)


✅ CONTRASTIVE_ROOT set to: /gpfs/commons/home/atalukder/Contrastive_Learning


In [2]:
def evaluate_tissue_binary_pm1_logitdelta_per_tissue_exonloop(
    gt_file: str,            # wide GT with {-1,0,+1} and 'logit_mean_psi'
    pred_file: str,          # predictions (PSI; % OK)
    model_name: str = "",
    filter_pred: bool = False,
    tissue_subset: list | None = None
) -> pd.DataFrame:
    """
    Binary tissue-specific evaluation:
      - Uses GT ∈ {-1,0,+1}; drops 0 and NaN; maps (-1 -> 0, +1 -> 1)
      - Score = Δlogit = logit(PSI_pred) − logit(mean_psi)
      - Hard label at 0: (Δlogit > 0) → 1
      - Aggregates per tissue across all exons in one exon-wise pass
    """
    merged, tissue_cols = load_and_align_for_delta_logit(
        gt_file=gt_file,
        pred_file=pred_file,
        require_cols=["logit_mean_psi"]
    )
    if tissue_subset is not None:
        tissue_cols = [t for t in tissue_cols if t in tissue_subset]

    # per-tissue accumulators
    acc = {t: {"y_true": [], "y_score": [], "y_pred": []} for t in tissue_cols}
    
    valid_exon = 0
    for _, row in merged.iterrows():
        # pull vectors for this exon (drop 0/NA, map -1/+1 -> 0/1)
        if not filter_pred:
            y_true_bin, y_psi, valid_mask = pull_pm1_vectors_from_row(row, tissue_cols)
        else:
            y_true_bin, y_psi, valid_mask = pull_pm1_vectors_from_row_predfilter(row, tissue_cols)
        if y_true_bin.size == 0:
            continue
        
        valid_exon+=1
        # per-position Δlogit relative to this exon’s baseline
        muL = float(row["logit_mean_psi"])
        dlogit = delta_logit_scores(y_psi, muL)  # higher ⇒ more +1

        # hard preds at Δlogit=0
        y_pred = (dlogit > 0).astype(int)

        # scatter back to tissues using the mask’s positions
        pos = 0
        for j, t in enumerate(tissue_cols):
            if valid_mask[j]:
                acc[t]["y_true"].append(int(y_true_bin[pos]))
                acc[t]["y_score"].append(float(dlogit[pos]))
                acc[t]["y_pred"].append(int(y_pred[pos]))
                pos += 1

    # compute metrics per tissue
    rows = []
    for t in tissue_cols:
        yt = np.asarray(acc[t]["y_true"], dtype=int)
        ys = np.asarray(acc[t]["y_score"], dtype=float)
        yp = np.asarray(acc[t]["y_pred"], dtype=int)
        if yt.size == 0 or np.unique(yt).size < 2 or not np.isfinite(ys).all():
            rows.append({"tissue": t, "model": model_name, "n": int(yt.size),
                         "accuracy": np.nan, "precision": np.nan, "recall": np.nan, "f1": np.nan,
                         "auroc": np.nan, "auprc": np.nan})
            continue
        m = binary_metrics(yt, yp, ys)
        m.update({"tissue": t, "model": model_name, "n": int(yt.size)})
        rows.append(m)

    return pd.DataFrame(rows).sort_values(["tissue","model"]).reset_index(drop=True)

def extract_valid_vectors(row, tissue_cols, gt_suffix="_gt", pred_suffix="_pred"):
    """Return (gt, pred, valid_mask) arrays for a merged exon row."""
    g = pd.to_numeric(row[[f"{t}{gt_suffix}" for t in tissue_cols]], errors="coerce").to_numpy(float)
    p = pd.to_numeric(row[[f"{t}{pred_suffix}" for t in tissue_cols]], errors="coerce").to_numpy(float)
    valid = np.isfinite(g) & np.isfinite(p)
    return g, p, valid

def evaluate_tissue_triclass_margin_per_tissue_exonloop(
    gt_file: str,
    pred_file: str,
    model_name: str = "",
    tissue_subset: list | None = None,
    margin_psi: float = 0.10,
    psi_bar_col: str = "mean_psi",
    use_logit_thresholds: bool = False,
    eps: float = 1e-6
) -> pd.DataFrame:
    """
    Three-class tissue-specific evaluation aligned to GT rule:
      GT: psi_pred > psi_bar + m -> +1; psi_pred < psi_bar - m -> -1; else 0.
      If use_logit_thresholds = True, we classify via Δlogit using per-exon τ_pos/τ_neg
      derived from the same ±m band (exactly equivalent to the PSI rule).
    Metrics: accuracy, macro-F1, weighted-F1 per tissue.
    """

    import numpy as np
    import pandas as pd
    from sklearn.metrics import accuracy_score, f1_score

    # helper functions reused
    def sigmoid(z): return 1.0 / (1.0 + np.exp(-z))
    def logit(p):   return np.log(p) - np.log1p(-p)

    # --- load data ---
    merged, tissue_cols = load_and_align_for_delta_logit(
        gt_file=gt_file,
        pred_file=pred_file,
        require_cols=["logit_mean_psi"]
    )
    if psi_bar_col not in merged.columns:
        merged[psi_bar_col] = sigmoid(merged["logit_mean_psi"].astype(float))
    if tissue_subset is not None:
        tissue_cols = [t for t in tissue_cols if t in tissue_subset]

    acc = {t: {"y_true": [], "y_pred": []} for t in tissue_cols}

    # --- iterate over exons ---
    for _, row in merged.iterrows():
        # use existing helper for vector extraction
        g, p, valid = extract_valid_vectors(row, tissue_cols)
        if not np.any(valid):
            continue

        psi_bar = float(row[psi_bar_col])
        psi_bar_clip = np.clip(psi_bar, eps, 1 - eps)
        muL = float(row["logit_mean_psi"])

        if use_logit_thresholds:
            # convert PSI thresholds to Δlogit thresholds (exact)
            psi_hi = np.clip(psi_bar + margin_psi, eps, 1 - eps)
            psi_lo = np.clip(psi_bar - margin_psi, eps, 1 - eps)
            tau_pos = logit(psi_hi) - muL
            tau_neg = logit(psi_lo) - muL
            dlogit = delta_logit_scores(p[valid], muL)
            # dlogit = compute_dlogit(p[valid], muL)

            y_pred = np.where(
                dlogit > tau_pos,  1,
                np.where(dlogit < tau_neg, -1, 0)
            )
        else:
            # direct PSI-based classification
            pv = p[valid]
            y_pred = np.where(
                pv > psi_bar + margin_psi,  1,
                np.where(pv < psi_bar - margin_psi, -1, 0)
            )

        # --- scatter to tissues ---
        pos = 0
        for j, t in enumerate(tissue_cols):
            if valid[j]:
                acc[t]["y_true"].append(int(g[j]))
                acc[t]["y_pred"].append(int(y_pred[pos]))
                pos += 1

    # --- compute metrics per tissue ---
    rows = []
    for t in tissue_cols:
        yt = np.asarray(acc[t]["y_true"], dtype=int)
        yp = np.asarray(acc[t]["y_pred"], dtype=int)
        if yt.size == 0 or np.unique(yt).size < 2:
            rows.append({
                "tissue": t, "model": model_name, "n": int(yt.size),
                "accuracy": np.nan, "macro_f1": np.nan, "weighted_f1": np.nan
            })
            continue

        rows.append({
            "tissue": t,
            "model": model_name,
            "n": int(yt.size),
            "accuracy": accuracy_score(yt, yp),
            "macro_f1": f1_score(yt, yp, average="macro"),
            "weighted_f1": f1_score(yt, yp, average="weighted")
        })

    return pd.DataFrame(rows).sort_values(["tissue", "model"]).reset_index(drop=True)


In [3]:
# # --- Paths you likely already have ---
GT_CLASS3_WIDE = f"{DATA_BASE}/test_cassette_exons_with_binary_labels_ExonBinPsi.csv"  # {-1,0,+1} wide

def evaluate_tissueSpecific_classification(result_file_name: str, model_user_name_norun: str, filter_pred: bool = False):
    """
    Tissue-specific classification evaluation.
    1️⃣ Binary: GT {-1,+1} only (drops 0/NA) via Δlogit threshold at 0.
    2️⃣ Tri-class: GT {-1,0,+1} using ±margin_psi rule (Δlogit or PSI thresholds).

    Both produce per-tissue CSVs and print summary stats.
    Returns (combined_df, summary_text)
    """
    import numpy as np
    import pandas as pd

    # 1️⃣ Locate prediction file
    pred_file = get_prediction_file(ROOT_RESULTS, result_file_name)
    print(f"📂 Found predictions for {model_user_name_norun} → {pred_file}")

    # --------------------------------------------------------------------------
    # 2️⃣ Run Binary tissue-specific evaluation
    per_tissue_bin = evaluate_tissue_binary_pm1_logitdelta_per_tissue_exonloop(
        gt_file=GT_CLASS3_WIDE,
        pred_file=pred_file,
        model_name=model_user_name_norun,
        tissue_subset=None,
        filter_pred=filter_pred
    )

    out_bin_csv = f"{OUT_DIR}/{model_user_name_norun}_tissue_metrics_logitdelta_binary_{timestamp}.csv"
    per_tissue_bin.to_csv(out_bin_csv, index=False)
    print(f"✅ Saved binary tissue metrics → {out_bin_csv}")

    # --- Binary summary
    n_tissues = per_tissue_bin["tissue"].nunique()
    macro_bin = per_tissue_bin[["accuracy","precision","recall","f1","auprc", "auroc"]].mean(skipna=True).round(4)
    summary_bin = (
        f"\n🧪 {model_user_name_norun} (Binary)\n"
        f"   n_tissues : {n_tissues}\n" +
        "\n".join([f"   {k:<10}: {v:.4f}" for k,v in macro_bin.items()]) + "\n"
    )
    print(summary_bin)

    # --------------------------------------------------------------------------
    # 3️⃣ Run Tri-class tissue-specific evaluation
    per_tissue_tri = evaluate_tissue_triclass_margin_per_tissue_exonloop(
        gt_file=GT_CLASS3_WIDE,
        pred_file=pred_file,
        model_name=model_user_name_norun,
        tissue_subset=None,          # restrict to certain tissues if needed
        margin_psi=0.10,             # matches GT ±0.1 rule
        use_logit_thresholds=True    # set False to use PSI space thresholds
    )

    out_tri_csv = f"{OUT_DIR}/{model_user_name_norun}_tissue_metrics_triclass_{timestamp}.csv"
    per_tissue_tri.to_csv(out_tri_csv, index=False)
    print(f"✅ Saved tri-class tissue metrics → {out_tri_csv}")

    # --- Tri-class summary
    n_tissues_tri = per_tissue_tri["tissue"].nunique()
    macro_tri = per_tissue_tri[["accuracy","macro_f1","weighted_f1"]].mean(skipna=True).round(4)
    summary_tri = (
        f"\n🧬 {model_user_name_norun} (Tri-class)\n"
        f"   n_tissues : {n_tissues_tri}\n" +
        "\n".join([f"   {k:<10}: {v:.4f}" for k,v in macro_tri.items()]) + "\n"
    )
    print(summary_tri)

    # --------------------------------------------------------------------------
    # 4️⃣ Combined summary
    print(f"📊 {model_user_name_norun}: "
          f"Binary F1={macro_bin['f1']:.4f}, "
          f"Tri-class macroF1={macro_tri['macro_f1']:.4f}")

    combined_summary = summary_bin + "\n" + summary_tri

    # For backward compatibility with your existing code
    combined_df = pd.concat([
        per_tissue_bin.assign(eval_type="binary"),
        per_tissue_tri.assign(eval_type="triclass")
    ], ignore_index=True)

    return combined_df, combined_summary


In [4]:
import numpy as np
import pandas as pd
# (You will also need to import your 'load_and_align_for_delta_logit' and 'binary_metrics' functions)

def pull_change_vs_nochange_vectors_from_row(row: pd.Series, tissue_cols: list) -> tuple[np.ndarray, np.ndarray, np.ndarray]: # CHANGED name
    """
    From a merged row (has <tissue>_gt and <tissue>_pred):
      - Keep only GT in {-1, 0, 1} (ignore NaN)
      - Map ground truth: -1 -> 0 (No Change) and {0, 1} -> 1 (Change)
      - Return (y_true_bin, y_psi, mask_idx)
    """
    # CHANGED all logic in this function
    import numpy as np
    import pandas as pd

    g = pd.to_numeric(row[[f"{t}_gt" for t in tissue_cols]], errors="coerce").to_numpy(dtype="float64")
    p = pd.to_numeric(row[[f"{t}_pred" for t in tissue_cols]], errors="coerce").to_numpy(dtype="float64")

    # valid where GT is -1, 0, or 1, and pred is finite
    valid = np.isfinite(p) & np.isfinite(g) & ((g == -1) | (g == 0) | (g == 1))
    if not np.any(valid):
        return np.array([], dtype=int), np.array([], dtype=float), valid

    # Get all valid ground truth labels (-1, 0, 1)
    g_valid = g[valid]
    
    # Create new binary labels:
    # -1 -> 0 (Negative Class)
    # 0  -> 1 (Positive Class)
    # 1  -> 1 (Positive Class)
    y_true_bin = (g_valid != -1).astype(int) 
    
    y_psi = p[valid].astype(float)
    return y_true_bin, y_psi, valid


def evaluate_tissue_binary_change_vs_nochange_logitdelta_per_tissue_exonloop( # CHANGED name
    gt_file: str,            # wide GT with {-1,0,+1} and 'logit_mean_psi'
    pred_file: str,          # predictions (PSI; % OK)
    model_name: str = "",
    filter_pred: bool = False,
    tissue_subset: list | None = None
) -> pd.DataFrame:
    """
    Binary tissue-specific evaluation:
      - Uses GT ∈ {-1,0,+1}; maps (-1 -> 0) and (0, 1 -> 1) # CHANGED logic
      - Score = abs(PSI_pred - mean_psi)                   # CHANGED score
      - Hard label at 0.1: (abs(ΔPSI) > 0.1) → 1           # CHANGED prediction
      - Aggregates per tissue across all exons in one exon-wise pass
    """
    merged, tissue_cols = load_and_align_for_delta_logit(
        gt_file=gt_file,
        pred_file=pred_file,
        require_cols=["logit_mean_psi"]
    )
    if tissue_subset is not None:
        tissue_cols = [t for t in tissue_cols if t in tissue_subset]

    # per-tissue accumulators
    acc = {t: {"y_true": [], "y_score": [], "y_pred": []} for t in tissue_cols}
    
    valid_exon = 0
    for _, row in merged.iterrows():

        # pull vectors for this exon (map -1->0, 0/1->1) # CHANGED
        y_true_bin, y_psi, valid_mask = pull_change_vs_nochange_vectors_from_row(row, tissue_cols)
        
        if y_true_bin.size == 0:
            continue
        
        valid_exon+=1

        # --- CHANGED SCORING LOGIC ---
        # We now score based on abs(ΔPSI), which matches the problem definition.
        
        # 1. Get mean_psi from logit_mean_psi
        muL = float(row["logit_mean_psi"])
        mu_psi = 1 / (1 + np.exp(-muL)) # Inverse logit
        
        # 2. Calculate ΔPSI for each prediction
        delta_psi = y_psi - mu_psi

        # 3. The SCORE (for AUCs) is the magnitude of the change
        y_score = np.abs(delta_psi)

        # 4. The PRED (for Precision) matches the ground truth definition
        y_pred = (y_score > 0.1).astype(int)
        # --- END OF CHANGED SCORING LOGIC ---

        # scatter back to tissues using the mask’s positions
        pos = 0
        for j, t in enumerate(tissue_cols):
            if valid_mask[j]:
                acc[t]["y_true"].append(int(y_true_bin[pos]))
                acc[t]["y_score"].append(float(y_score[pos])) # CHANGED
                acc[t]["y_pred"].append(int(y_pred[pos]))   # CHANGED
                pos += 1

    # compute metrics per tissue
    rows = []
    for t in tissue_cols:
        yt = np.asarray(acc[t]["y_true"], dtype=int)
        ys = np.asarray(acc[t]["y_score"], dtype=float)
        yp = np.asarray(acc[t]["y_pred"], dtype=int)
        
        # Check for valid data (e.g., must have both 0s and 1s)
        if yt.size == 0 or np.unique(yt).size < 2 or not np.isfinite(ys).all():
            rows.append({"tissue": t, "model": model_name, "n": int(yt.size),
                         "accuracy": np.nan, "precision": np.nan, "recall": np.nan, "f1": np.nan,
                         "auroc": np.nan, "auprc": np.nan})
            continue
            
        m = binary_metrics(yt, yp, ys)
        m.update({"tissue": t, "model": model_name, "n": int(yt.size)})
        rows.append(m)

    return pd.DataFrame(rows).sort_values(["tissue","model"]).reset_index(drop=True)




In [5]:
# --- Paths you likely already have ---
GT_CLASS3_WIDE = f"{DATA_BASE}/test_cassette_exons_with_binary_labels_ExonBinPsi.csv"  # {-1,0,+1} wide
# (You will also need to have your get_prediction_file, load_and_align_for_delta_logit, 
#  binary_metrics, and the two new functions we just defined:
#  - evaluate_tissue_binary_change_vs_nochange_logitdelta_per_tissue_exonloop
#  - evaluate_tissue_binary_pm1_logitdelta_per_tissue_exonloop
#  ...available in this script's scope)

def evaluate_tissueSpecific_classification_changeNOchange(
    result_file_name: str, 
    model_user_name_norun: str,
    filter_pred: bool = False
):
    """
    Tissue-specific classification evaluation.
    
    Can run one of two binary modes based on 'analysis_type':
    1️⃣ 'change_vs_nochange': GT {-1, 0, +1} -> {0, 1, 1} (Change vs. No Change)
    2️⃣ 'up_vs_down':         GT {-1, +1} only (drops 0/NA) (Up vs. Down)

    Produces per-tissue CSVs and prints summary stats.
    """
    import numpy as np
    import pandas as pd

    # 1️⃣ Locate prediction file
    pred_file = get_prediction_file(ROOT_RESULTS, result_file_name)
    print(f"📂 Found predictions for {model_user_name_norun} → {pred_file}")

    # --------------------------------------------------------------------------
    # 2️⃣ Run the selected Binary tissue-specific evaluation
    
    # CHANGED: Added if/elif block to select the analysis
    
   
    print("\nRunning: Binary (Change vs. No Change) analysis")
    per_tissue_metrics = evaluate_tissue_binary_change_vs_nochange_logitdelta_per_tissue_exonloop(
        gt_file=GT_CLASS3_WIDE,
        pred_file=pred_file,
        model_name=model_user_name_norun,
        tissue_subset=None,
        filter_pred=filter_pred
    )
    out_bin_csv = f"{OUT_DIR}/{model_user_name_norun}_tissue_metrics_binary_change_vs_nochange_{timestamp}.csv"
    summary_label = "Binary (Change vs. No Change)"

    per_tissue_metrics.to_csv(out_bin_csv, index=False) # CHANGED: uses new out_bin_csv variable
    print(f"✅ Saved binary tissue metrics → {out_bin_csv}")

    # --- Binary summary ---
    n_tissues = per_tissue_metrics["tissue"].nunique() # CHANGED: uses new per_tissue_metrics variable
    macro_bin = per_tissue_metrics[["accuracy","precision","recall","f1","auprc", "auroc"]].mean(skipna=True).round(4) # CHANGED
    
    summary_bin = (
        f"\n🧪 {model_user_name_norun} ({summary_label})\n" # CHANGED: uses new summary_label
        f"   n_tissues : {n_tissues}\n" +
        "\n".join([f"   {k:<10}: {v:.4f}" for k,v in macro_bin.items()]) + "\n"
    )
    print(summary_bin)
    return per_tissue_metrics, summary_bin
    

In [6]:
def plot_classification_summary(results_df, metric="auprc", out_name=None):
    plt.figure(figsize=(10, 5))
    sns.boxplot(
        data=results_df,
        x="expression_type",
        y=metric,
        hue="model",
        showfliers=False
    )
    plt.ylabel(metric.upper())
    plt.title(f"Per-exon {metric.upper()} distribution across models (HIGH/LOW exons)")
    plt.legend(loc="upper right", fontsize=8)
    plt.tight_layout()
    if out_name:
        out_path = f"{OUT_DIR}/{out_name}.png"
        plt.savefig(out_path, dpi=300, bbox_inches="tight")
        print(f"📊 Saved figure → {out_path}")
    plt.show()

In [7]:
time_stamp = time.strftime("%Y_%m_%d__%H_%M_%S", time.localtime())
SUMMARY_TXT = f"{OUT_DIR}/ExonSpecific_classification_summary_{time_stamp}.txt"
summary_lines = ["===== MODEL CLASSIFICATION SUMMARY ====="]

# --- Model 1 ---
result_file_name1 =  "exprmnt_2025_11_05__01_50_41" # EMPRAIPsi_TS_noCL_300bp_rerun_codeChange_2025_11_05__01_50_41
model1_user_name_norun = 'TS_noCL_300bp_rerun_codeChange'
# --- Model 2 ---
result_file_name2 = "exprmnt_2025_11_05__01_52_25" # EMPRAIPsi_TS_CLSwpd_300bp_10Aug_rerun_codeChange_2025_11_05__01_52_25
model2_user_name_norun = 'TS_CLSwpd_300bp_10Aug'

In [ ]:
per_tissue_metrics, summary_bin = evaluate_tissueSpecific_classification_changeNOchange(
    result_file_name=result_file_name1,
    model_user_name_norun=model1_user_name_norun, 
)
summary_lines += [summary_bin]

per_tissue_metrics, summary_bin = evaluate_tissueSpecific_classification_changeNOchange(
    result_file_name=result_file_name2,
    model_user_name_norun=model2_user_name_norun, 
)
summary_lines += [summary_bin]

with open(SUMMARY_TXT, "w") as f:
    f.write("\n".join(summary_lines))

In [8]:
# tissue specific classification evaluation

# --- Model 1 ---
df_model1, txt_model1 = evaluate_tissueSpecific_classification(result_file_name1, model1_user_name_norun, filter_pred=False)
summary_lines += [txt_model1]

# --- Model 2 ---
df_model2, txt_model2 = evaluate_tissueSpecific_classification(result_file_name2, model2_user_name_norun, filter_pred=False)
summary_lines += [txt_model2]

with open(SUMMARY_TXT, "w") as f:
    f.write("\n".join(summary_lines))

📂 Found predictions for TS_noCL_300bp_rerun_codeChange → /gpfs/commons/home/atalukder/Contrastive_Learning/files/results/exprmnt_2025_11_05__01_50_41/ensemble_evaluation_from_valdiation/test_set_evaluation/tsplice_final_predictions_all_tissues.tsv
✅ Saved binary tissue metrics → /gpfs/commons/home/atalukder/Contrastive_Learning/files/results/../classification_eval/TS_noCL_300bp_rerun_codeChange_tissue_metrics_logitdelta_binary_20251105-182157.csv

🧪 TS_noCL_300bp_rerun_codeChange (Binary)
   n_tissues : 112
   accuracy  : 0.5457
   precision : 0.2014
   recall    : 0.4872
   f1        : 0.2294
   auprc     : 0.2739
   auroc     : 0.5485

✅ Saved tri-class tissue metrics → /gpfs/commons/home/atalukder/Contrastive_Learning/files/results/../classification_eval/TS_noCL_300bp_rerun_codeChange_tissue_metrics_triclass_20251105-182157.csv

🧬 TS_noCL_300bp_rerun_codeChange (Tri-class)
   n_tissues : 112
   accuracy  : 0.0895
   macro_f1  : 0.0756
   weighted_f1: 0.0800

📊 TS_noCL_300bp_rerun_co

In [ ]:
def plot_tissue_classification_box(results_df, metric="auprc", out_name=None,
                                       model_order=None, palette=None, show_points=True):
    """
    Boxplot of per-tissue metrics across models (no pre-aggregation).
    Expects columns: ['tissue', 'model', <metric>].
    """
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns

    # Keep only finite metric values; no groupby needed
    df = results_df.replace([np.inf, -np.inf], np.nan).dropna(subset=[metric, "model"])

    plt.figure(figsize=(10, 5))
    ax = sns.boxplot(
        data=df,
        x="model",
        y=metric,
        order=model_order,
         hue="model",
        showfliers=False
    )

    if show_points:
        sns.stripplot(
            data=df,
            x="model",
            y=metric,
            order=model_order,
            dodge=False,
            jitter=0.12,
            size=3,
            alpha=0.65,
            color="k"  # keep legend clean
        )

    if metric.lower() in {"auprc", "auroc"}:
        plt.ylim(0, 1)

    plt.ylabel(metric.upper())
    plt.title(f"Per-tissue {metric.upper()} distribution across models")
    plt.xlabel("model")
    plt.tight_layout()

    if out_name:
        out_path = f"{OUT_DIR}/{out_name}.png"
        plt.savefig(out_path, dpi=300, bbox_inches="tight")
        print(f"📊 Saved figure → {out_path}")

    plt.show()



In [ ]:
combined_all = pd.concat([df_sota, df_model1, df_model2], ignore_index=True)
combined_all.to_csv(f"{OUT_DIR}/combined_tissue_classification_summary.csv", index=False)
print(f"✅ Combined summary saved → {OUT_DIR}/combined_tissue_classification_summary.csv")

# Use the tissue plotter (not the exon HIGH/LOW plotter)
# plot_tissue_classification_summary(combined_all, metric="auprc", out_name="AUPRC_tissue_SOTA_vs_models")
# plot_tissue_classification_summary(combined_all, metric="auroc", out_name="AUROC_tissue_SOTA_vs_models")

plot_tissue_classification_box(combined_all, metric="auprc", out_name="AUPRC_tissue_box_SOTA_vs_models")
plot_tissue_classification_box(combined_all, metric="auroc", out_name="AUROC_tissue_box_SOTA_vs_models")
